# Termination Conditions

### Question: What is a termination condition?
Answer: When agents are running in a team, a Termination Condition defines the criteria for halting a conversation or process. 

It is crucial for ensuring that runs are terminated at the right time, rather than running endlessly.

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=1,
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback for every message. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

In [2]:
max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

await Console(round_robin_team.run_stream(task="Describe a futuristic city in 50 words"))

---------- user ----------
Describe a futuristic city in 50 words
---------- primary ----------
In a futuristic city, soaring towers covered in vertical gardens merge with the sky. Autonomous vehicles glide silently along transparent streets, while drones buzz above. Solar panels and wind turbines power eco-friendly habitats. Holographic interfaces connect inhabitants, blending reality with augmented layers. Bioluminescent flora lights pathways beneath a shimmering dome.
---------- critic ----------
This description effectively paints a vivid picture of a futuristic city. However, one area for improvement is adding a sense of community or human activity to breathe life into this vision. Consider mentioning how people interact within this environment or how these advanced technologies enhance their daily lives. 

Overall, it's a captivating depiction, and with minor enhancements, it could become even more engaging.


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Describe a futuristic city in 50 words', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=66), content='In a futuristic city, soaring towers covered in vertical gardens merge with the sky. Autonomous vehicles glide silently along transparent streets, while drones buzz above. Solar panels and wind turbines power eco-friendly habitats. Holographic interfaces connect inhabitants, blending reality with augmented layers. Bioluminescent flora lights pathways beneath a shimmering dome.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=113, completion_tokens=73), content="This description effectively paints a vivid picture of a futuristic city. However, one area for improvement is adding a sense of community or human activity to breathe life into this vision. Consider mentioning how people interact within this envi

# Combining Termination Conditions in Autogen AgentChat

In Autogen AgentChat, termination conditions can be combined using the logical `AND (&)` and `OR (|)` operators. This allows for more complex termination logic to control when a conversation should stop.

## Example: Combining Termination Conditions  

In the following example, we create a team where the conversation stops if **either** of the following conditions is met:  
1. **A maximum of 10 messages** have been exchanged.  
2. **The critic agent approves a message** by responding with `"APPROVE"`.

By using the `|` (OR) operator, the conversation will terminate as soon as **one of these conditions is satisfied**.


In [4]:

# Define termination conditions
max_msg_termination = MaxMessageTermination(max_messages=10)
critic_approval_termination = TextMentionTermination(text="APPROVE")

# Combine conditions using OR (|)
combined_termination = max_msg_termination | critic_approval_termination

# Create a round-robin team with the combined termination condition
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=combined_termination)

# Execute the conversation task
await Console(round_robin_team.run_stream(task="Provide a summary of the latest advancements in AI."))


---------- user ----------
Provide a summary of the latest advancements in AI.


---------- primary ----------
Recent advancements in AI have focused on several key areas:

1. **Large Language Models (LLMs):** Models like OpenAI's GPT-4 and others have advanced in generating text that is increasingly coherent and context-aware, with applications ranging from content creation to conversational agents.

2. **Image and Video Generation:** AI systems are now capable of creating highly realistic images and videos, evident in advancements like DALL-E, which can generate images from textual descriptions.

3. **Multimodal AI:** Systems that integrate and process multiple types of data, such as text, image, and sound, are improving, allowing for more comprehensive understanding and interaction with complex data sources.

4. **Reinforcement Learning:** Enhanced algorithms have enabled AI to master complex environments with better strategies, evident in applications from robotics to game playing.

5. **AI in Healthcare:** AI continues to make strides in diagnostic tools, pred

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Provide a summary of the latest advancements in AI.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=429, completion_tokens=288), content="Recent advancements in AI have focused on several key areas:\n\n1. **Large Language Models (LLMs):** Models like OpenAI's GPT-4 and others have advanced in generating text that is increasingly coherent and context-aware, with applications ranging from content creation to conversational agents.\n\n2. **Image and Video Generation:** AI systems are now capable of creating highly realistic images and videos, evident in advancements like DALL-E, which can generate images from textual descriptions.\n\n3. **Multimodal AI:** Systems that integrate and process multiple types of data, such as text, image, and sound, are improving, allowing for more comprehensive understanding and interaction with complex data sources.\n\n4. **Reinforcement L

## Using AND (`&`) for Combined Termination Conditions in Autogen AgentChat  

In addition to using the OR (`|`) operator, termination conditions can also be combined using the AND (`&`) operator. This ensures that the conversation will **only stop when all specified conditions are met**.

It can be written like
`combined_termination = max_msg_termination & text_termination`


> You can also create custom termination conditions and details can be found here https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html